## Решение соревнования Journey to Springfield

Имя команды на лидерборде: Dmitrii_Shumilin_263849669
Результат на лидерборде: 0.99787

In [1]:
# General libs
import pandas as pd
import numpy as np
import random
from tqdm.notebook import tqdm
from pathlib import Path
from datetime import datetime
import pickle

# Libs for plotting
from PIL import Image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from torchsummary import summary

# Pytorch
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

# Sklearn
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#### Подготовка данных

Зафиксируем сиды и укажем устройство на которое помещать информацию при расчете

In [2]:
SEED = 2020
DEVICE = torch.device("cuda")
MODES = ['train', 'val', 'test']

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Для простоты до создания датасета будем использовать пути файлов, а не сами файлы. Ниже подгрузим пути, разделим на тренировочную выборку и валидационную в соотношении 4:1 и сделаем распределение классов в выборках одинаковым.

In [ ]:
TRAIN_DIR = Path('train/simpsons_dataset/')
TEST_DIR = Path('testset/testset/')
train_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))

train_val_labels = [path.parent.name for path in train_files]
train_files, val_files = train_test_split(train_files, test_size=0.2, stratify=train_val_labels, shuffle=True)
print(len(train_files), len(val_files))

Поскольку известно, что существует несколько классов с числом объектов в датасете меньшим чем N, найдем их и добавим случайные подвыборки из Тренировочной выборки для этих классов в Тренировочный датасет. Цель - сделать 150 объектов во всех классах, где объектов меньше 150.

In [ ]:
label_series = pd.Series(list(map(lambda x: x.parent.name, train_files)) )
label_comp = label_series.value_counts()[label_series.value_counts()<150]

to_dict = list(map(lambda x, y: [x.parent.name,y], train_files,train_files))

dict_map = dict()
for name, path in to_dict:
    if name in dict_map:
        dict_map[name].append(path)
    else:
        dict_map[name] = [path]

for name, num in zip(label_comp.index, label_comp):
    source = dict_map[name]
    to_upsample = random.choices(source, k=150-num)
    train_files.extend(to_upsample)
    
print(len(train_files), len(val_files))

#### Создание загрузчиков данных

Создадим класс аугментации для добавления к изображению гаусовского шума

In [ ]:
class GaussianNoise():
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

При сборе датасета для тренировочной выборки будем создавать много аугментированных изображений на лету, а так же нормализировать набор данных. Нормализировать будем при помощи известных наборов средних величин и стандартного отколонения для Imagenet набора данных, так как предполагается использование предобученной на этом наборе данных модели.

In [ ]:
class SimpsonsDataset(Dataset):
    def __init__(self, files, mode):
        super().__init__()
        self.files = sorted(files)
        self.mode = mode

        if self.mode not in MODES:
            print(f"{self.mode} is not correct; correct modes: {MODES}")
            raise NameError

        self.len_ = len(self.files)
     
        self.label_encoder = LabelEncoder()

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)
            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)

                      
    def __len__(self):
        return self.len_
      
    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image
  
    def __getitem__(self, index):
        if self.mode == 'train': 
            transform = transforms.Compose([
                transforms.Resize(size=(224, 224)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(degrees=15),
                transforms.RandomApply([transforms.ColorJitter(hue=0.1, saturation=0.1)], p=0.35),
                transforms.RandomApply([transforms.RandomAffine(0, translate=None, scale=None, shear= (-10, 10) ,resample=False, fillcolor=0)], p=0.5),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                transforms.RandomApply([GaussianNoise(0, 0.25)], p=0.25)
            ])
        else:
            transform = transforms.Compose([
                transforms.Resize(size=(224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
            ])
        x = self.load_sample(self.files[index])
        x = transform(x)
        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y

Создадим загрузчики данных с размером батча 128 изображений.

In [ ]:
train_dataset = SimpsonsDataset(train_files, mode='train')
val_dataset = SimpsonsDataset(val_files, mode='val')
train_loader = DataLoader(train_dataset, batch_size=128, num_workers=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, num_workers=8, shuffle=False)
loaders = {
    "train": train_loader,
    "valid": val_loader,
}

#### Модель

Для выполнения задачи будем использовать Resnet50 предобученную на Imagenet.

In [ ]:
resnet50 = models.resnet50(pretrained=True)

Посмотрим на ее структурные блоки

In [ ]:
resnet50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Ниже отключим вычисление градиентов для всей сети и вместо последнего линейного классификатора установим собственный линейный слой с выходным в 42. Градиенты для нового слоя автоматически включены.

In [ ]:
def set_parameter_requires_grad(model):
    for param in model.parameters():
        param.requires_grad = False
        
set_parameter_requires_grad(resnet50)
resnet50.fc = nn.Linear(2048, 42)
resnet50.to(DEVICE)
summary(resnet50, (3, 224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
# for param in effnet.parameters():
#   print(param.requires_grad)

Для начального приближения выберем оптимайзер AdamW с большим learning rate. На этом этапе это не принципиально.

In [ ]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(resnet50.parameters(), lr=0.01, betas=(0.9, 0.999), weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, min_lr=0, verbose=True)

#### Обучение сети

In [ ]:
def log_in_file(file_name, model, optimizer, scheduler, loss,
                epoch, epoch_loss_train, epoch_loss_val,
                epoch_accuracy_train, epoch_accuracy_val,
                epoch_f1_val):
    """
    Функция для записи результатов расчета в txt файл
    """
    with open(f'{file_name}.txt', 'a') as the_file:
        if epoch==1:
            the_file.write(f"Starting time to logging: {datetime.now().strftime('%Y%m%d-%H%M%S')} \n")
            the_file.write(f"Network architercure: {model} \n")
            the_file.write(f"Oprimizer: {optimizer} \n")
            the_file.write(f"Scheduler: {scheduler} \n")
            the_file.write(f"Scheduler parameters: {scheduler.state_dict()} \n")
            the_file.write(f"Loss function: {criterion} \n")
            the_file.write(f"SEED for learning: {SEED} \n\n")
            the_file.write(f"Learning process: \n")


        the_file.write(f'Epoch: {epoch}, Train loss: {epoch_loss_train}, Val loss: {epoch_loss_val} \n')
        the_file.write(f'Epoch: {epoch}, Train accuracy {epoch_accuracy_train}, Val accuracy: {epoch_accuracy_val} \n')
        the_file.write(f'Epoch: {epoch}, F1 Val score: {epoch_f1_val} \n\n')
    return

Декомпозируем большую функцию тренировочной петли в несколько мелких:
* Обучение на батче
* Получение ответов на валидационном батче
* Расчет метрик
* Основная петля обучения

In [ ]:
def train_batch(model, X, y, criterion, optimizer):
    model.train(True)
    X = X.to(DEVICE)
    y = y.to(DEVICE)

    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    pred = torch.argmax(output, dim=1).cpu().numpy()
    return loss, pred

In [ ]:
def val_batch(model, X, y, criterion):
    model.train(False)
    X = X.to(DEVICE)
    y = y.to(DEVICE)
    with torch.set_grad_enabled(False):
        output = model(X)
        loss = criterion(output, y)
        pred = torch.argmax(output, dim=1).cpu().numpy()
    return loss, pred

Из метрик будем получать точность и F1 меру

In [ ]:
def get_metrics(pred, y_true):
    accuracy = metrics.accuracy_score(y_true.ravel(), pred.ravel())
    f1 = metrics.f1_score(y_true.ravel(), pred.ravel(), average='macro')
    return accuracy, f1

В Функции основной петли реализовано обучение, логировоание и сохраниение лучшей модели (при положительном существующем флаге Save) на основе достижения лучшего результата.

In [ ]:
def train_loop(model, loaders, criterion, optimizer, scheduler, EPOCHES , save=False):
    train_loss = []
    train_f1_score = []
    train_accuracy = []
    val_loss = []
    val_f1_score = []
    val_accuracy = []
    best_f1_score = 0


    for epoch in tqdm(range(1, EPOCHES+1)):
        train_loss_on_epoch = 0
        val_loss_on_epoch = 0
        train_predict = np.array([])
        train_true = np.array([])
        val_predict = np.array([])
        val_true = np.array([])
        i=0
        for X, y in tqdm(loaders['train']):
            loss, pred = train_batch(model, X, y, criterion, optimizer)
            train_loss_on_epoch += loss
            i += 1
            train_predict = np.hstack((train_predict, pred))
            train_true = np.hstack((train_true, y.numpy()))

        train_loss.append(train_loss_on_epoch/i)
        tr_acc, tr_f1 = get_metrics(train_predict, train_true)
        train_f1_score.append(tr_f1)
        train_accuracy.append(tr_acc)
        

        j=0
        for X, y in tqdm(loaders['valid']):
            loss, pred = val_batch(model, X, y, criterion)
            val_loss_on_epoch += loss
            j += 1
            val_predict = np.hstack((val_predict, pred))
            val_true = np.hstack((val_true, y.numpy()))

        val_loss.append(val_loss_on_epoch/j)
        v_acc, v_f1 = get_metrics(val_predict, val_true)
        val_f1_score.append(v_f1)
        val_accuracy.append(v_acc)
        print(f'Epoch: {epoch}, Train loss: {train_loss_on_epoch/i}, Train accuracy {tr_acc}, Train f1 score {tr_f1}')
        print(f'Epoch: {epoch}, Val loss: {val_loss_on_epoch/j}, Val accuracy {v_acc}, Val f1 score {v_f1}')
        print(f'Best_f1_score {best_f1_score}')

        scheduler.step(val_loss_on_epoch/j)

        log_in_file('effnet_logs', model, optimizer, scheduler, criterion,
                epoch, train_loss_on_epoch, val_loss_on_epoch,
                tr_acc, v_acc,
                v_f1)


        if (best_f1_score<v_f1):
            best_f1_score = v_f1
            if save:
                torch.save(model,'resnet_f1')

    history = {
        'loss':[train_loss, val_loss],
        'accuracy':[train_accuracy, val_accuracy],
        'f1_score':[train_f1_score, val_f1_score]
    }
    return history

Поскольку полносвязный слой никогда не обучался, имеет смысл оставить обучаться только его на несколько (в большенстве случаев произвольное колличество) эпох, для того, что бы большая ошибка, и соответственно, градиенты не возмущали уже хорошо подобранные веса остальной модели.

In [ ]:
history = train_loop(resnet50, loaders, criterion, optimizer, scheduler, 15)


Epoch: 1, Train loss: 5.0919623374938965, Train accuracy 0.3236521096266568, Train f1 score 0.2598658820819092
Epoch: 1, Val loss: 1.5531527996063232, Val accuracy 0.6056842608072606, Val f1 score 0.4050162908932752
Best_f1_score 0



Epoch: 2, Train loss: 1.4144502878189087, Train accuracy 0.6212705285948144, Train f1 score 0.5786881847431365
Epoch: 2, Val loss: 1.3735707998275757, Val accuracy 0.6252686887986625, Val f1 score 0.4845320377197879
Best_f1_score 0.4050162908932752



Epoch: 3, Train loss: 1.2944839000701904, Train accuracy 0.6484659660981148, Train f1 score 0.6240866950918585
Epoch: 3, Val loss: 1.4162958860397339, Val accuracy 0.6333890613804634, Val f1 score 0.47139553936071993
Best_f1_score 0.4845320377197879



Epoch: 4, Train loss: 1.1615095138549805, Train accuracy 0.6788826107620003, Train f1 score 0.664576480368431
Epoch: 4, Val loss: 1.3417304754257202, Val accuracy 0.6448531167900645, Val f1 score 0.5033928410191697
Best_f1_score 0.4845320377197879



Epoch: 5, Train loss: 1.1185762882232666, Train accuracy 0.6875429054232455, Train f1 score 0.684333011174782
Epoch: 5, Val loss: 1.406315803527832, Val accuracy 0.6159541437783616, Val f1 score 0.4786282626207441
Best_f1_score 0.5033928410191697


...


Epoch: 11, Train loss: 1.0108075141906738, Train accuracy 0.7267254580979036, Train f1 score 0.7285191057938656
Epoch: 11, Val loss: 1.2432520389556885, Val accuracy 0.6852161452113685, Val f1 score 0.5155729428094212
Best_f1_score 0.55515439665786



Epoch: 12, Train loss: 0.9986816048622131, Train accuracy 0.7258277446269208, Train f1 score 0.7280349574596965
Epoch: 12, Val loss: 1.250099539756775, Val accuracy 0.6775734416049678, Val f1 score 0.5516286702619141
Best_f1_score 0.55515439665786



Epoch: 13, Train loss: 1.0223987102508545, Train accuracy 0.7231874108887363, Train f1 score 0.7196046787744284
Epoch: 13, Val loss: 1.3249554634094238, Val accuracy 0.6854549796990685, Val f1 score 0.5574659186992837
Best_f1_score 0.55515439665786
Epoch    13: reducing learning rate of group 0 to 1.0000e-03.



Epoch: 14, Train loss: 0.754623532295227, Train accuracy 0.7848656070127265, Train f1 score 0.7986250129274198
Epoch: 14, Val loss: 1.0038518905639648, Val accuracy 0.7387150704561739, Val f1 score 0.6134261778259498
Best_f1_score 0.5574659186992837



Epoch: 15, Train loss: 0.7208789587020874, Train accuracy 0.7941067750963722, Train f1 score 0.8084633381274813
Epoch: 15, Val loss: 0.9766768217086792, Val accuracy 0.7379985669930738, Val f1 score 0.619183422374612
Best_f1_score 0.6134261778259498



При установившемся резульате обучения полносвязного слоя, позволим уже последнему блоку слоев (avgpool и layer4) обучиться так же на 15 итераций. Доучивание сети частями выполнялось при помощи обновления ячеек выше. После этого полная сеть будет обучаться на данных. Для этого затребуем вычисление градиентов, переобозначим оптимизатор и scheduler для сброса предыдущих состояний. Для не сильного изменения параметров сети уменьшим на 10 learning rate.

In [ ]:
def set_parameter_requires_grad(layers):
    for param in layers.parameters():
        param.requires_grad = True

set_parameter_requires_grad(resnet50)

In [ ]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(resnet50.parameters(), lr=0.0001, betas=(0.9, 0.999), weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, min_lr=0, verbose=True)
history = train_loop(resnet50, loaders, criterion, optimizer, scheduler, 60, save=True)


Epoch: 1, Train loss: 0.0017511051846668124, Train accuracy 0.9995247399271268, Train f1 score 0.9996895481502721
Epoch: 1, Val loss: 0.1364259272813797, Val accuracy 0.9782660616192979, Val f1 score 0.9499206551542679
Best_f1_score 0



Epoch: 2, Train loss: 0.0006908036884851754, Train accuracy 0.9998415799757089, Train f1 score 0.9999233866706599
Epoch: 2, Val loss: 0.13554218411445618, Val accuracy 0.9773107236684977, Val f1 score 0.949147449455341
Best_f1_score 0.9499206551542679



Epoch: 3, Train loss: 0.0010956873884424567, Train accuracy 0.9997887733009453, Train f1 score 0.9998253334990368
Epoch: 3, Val loss: 0.13570895791053772, Val accuracy 0.9775495581561977, Val f1 score 0.9500659493287018
Best_f1_score 0.9499206551542679



Epoch: 4, Train loss: 0.00104048871435225, Train accuracy 0.9996303532766542, Train f1 score 0.9998009824318866
Epoch: 4, Val loss: 0.13516080379486084, Val accuracy 0.9780272271315978, Val f1 score 0.946720459481589
Best_f1_score 0.9500659493287018



Epoch: 5, Train loss: 0.0009616569150239229, Train accuracy 0.9997887733009453, Train f1 score 0.9998416118844162
Epoch: 5, Val loss: 0.13303467631340027, Val accuracy 0.9780272271315978, Val f1 score 0.9465802886081504
Best_f1_score 0.9500659493287018


...


Epoch: 56, Train loss: 0.0005537676042877138, Train accuracy 0.9998943866504726, Train f1 score 0.9999451500820716
Epoch: 56, Val loss: 0.11591381579637527, Val accuracy 0.9799379030331979, Val f1 score 0.9538645860160261
Best_f1_score 0.9536339144180745



Epoch: 57, Train loss: 0.0004975839983671904, Train accuracy 0.9998943866504726, Train f1 score 0.9999451500820716
Epoch: 57, Val loss: 0.11606691777706146, Val accuracy 0.9789825650823979, Val f1 score 0.9529326950956755
Best_f1_score 0.9538645860160261



Epoch: 58, Train loss: 0.0006214100867509842, Train accuracy 0.9999471933252363, Train f1 score 0.9999558814315448
Epoch: 58, Val loss: 0.11522162705659866, Val accuracy 0.9792213995700979, Val f1 score 0.9536734243224638
Best_f1_score 0.9538645860160261



Epoch: 59, Train loss: 0.0005120540736243129, Train accuracy 0.9998943866504726, Train f1 score 0.9998823807828318
Epoch: 59, Val loss: 0.1144736185669899, Val accuracy 0.9782660616192979, Val f1 score 0.9521361210449655
Best_f1_score 0.9538645860160261



Epoch: 60, Train loss: 0.0005543223232962191, Train accuracy 1.0, Train f1 score 1.0
Epoch: 60, Val loss: 0.11469841003417969, Val accuracy 0.9792213995700979, Val f1 score 0.9526786287727592
Best_f1_score 0.9538645860160261



In [ ]:
# Для загрузки если колаб упал(
# resnet50 = torch.load('resnet_f1')

#### Делаем Submit на Kaggle

In [ ]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [ ]:
label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))

In [ ]:
test_dataset = SimpsonsDataset(test_files, mode="test")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)
probs = predict(resnet50, test_loader)


preds = label_encoder.inverse_transform(np.argmax(probs, axis=1))
test_filenames = [path.name for path in test_dataset.files]

In [ ]:
# my_submit = pd.read_csv("sample_submission.csv")
my_submit = pd.DataFrame({'Id': test_filenames, 'Expected': preds})
my_submit.head()

,Id,Expected
0,img0.jpg,nelson_muntz
1,img1.jpg,bart_simpson
2,img10.jpg,ned_flanders
3,img100.jpg,chief_wiggum
4,img101.jpg,apu_nahasapeemapetilon


In [ ]:
my_submit.to_csv('resnet50_all_layers.csv', index=False)

Вся необходимая информация по индентификации результата указана в начале файла.

Хорошего вам дня)